In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.1     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



[1] "students-order-brushing-1"

Load and inspect order data

In [8]:
order_data<-read.csv('order_brush_order.csv')
head(order_data)
str(order_data)

,orderid,shopid,userid,event_time
,<dbl>,<int>,<int>,<fct>
1,3.107658e+13,93950878,30530270,2019-12-27 00:23:03
2,3.111806e+13,156423439,46057927,2019-12-27 11:54:20
3,3.112336e+13,173699291,67341739,2019-12-27 13:22:35
4,3.112206e+13,63674025,149380322,2019-12-27 13:01:00
5,3.111708e+13,127249066,149493217,2019-12-27 11:37:55
6,3.111973e+13,173811070,116451780,2019-12-27 12:22:05


'data.frame':	222750 obs. of  4 variables:
 $ orderid   : num  3.11e+13 3.11e+13 3.11e+13 3.11e+13 3.11e+13 ...
 $ shopid    : int  93950878 156423439 173699291 63674025 127249066 173811070 107921853 178400128 147941492 164933170 ...
 $ userid    : int  30530270 46057927 67341739 149380322 149493217 116451780 166741763 61272835 10986763 135957741 ...
 $ event_time: Factor w/ 121064 levels "2019-12-27 00:00:00",..: 606 8499 10389 10001 8107 9121 19520 1638 8645 13830 ...


Filter for finding suspected shopid & userid

In [13]:
#Filter for same shopid & userid with more than 2 appearances
order_data_filtered<-order_data %>% 
  group_by(shopid,userid) %>% 
  filter(n()>2) %>% 
  arrange(shopid,userid)
#Separate date and time
order_data_filtered<-order_data_filtered %>% 
  separate(event_time,c('Date','Time'),sep=' ')
#Filter for same shopid & userid in 1 day transaction
order_data_filtered<-order_data_filtered %>%   
  group_by(shopid,userid,Date) %>% 
  filter(n()>2)
head(order_data_filtered)

orderid,shopid,userid,Date,Time
<dbl>,<int>,<int>,<chr>,<chr>
3.132229e+13,10159,214988798,2019-12-29,20:38:09
3.132245e+13,10159,214988798,2019-12-29,20:40:54
3.132340e+13,10159,214988798,2019-12-29,20:56:43
3.114129e+13,10287,103289694,2019-12-27,18:21:32
3.114912e+13,10287,103289694,2019-12-27,20:31:58
3.114818e+13,10287,103289694,2019-12-27,20:16:19


Inspect "1 day concentration ratio" for each pairs

In [14]:
order_subs1<-order_data_filtered %>% 
  count(shopid)
head(order_subs1)

shopid,userid,Date,n
<int>,<int>,<chr>,<int>
10159,214988798,2019-12-29,3
10287,103289694,2019-12-27,3
10287,198097381,2019-12-30,3
10287,214226569,2019-12-31,3
10402,77819,2019-12-28,3
10536,672345,2019-12-27,6


Combine shopid with more than 1 suspected userid

In [15]:
order_subs1<-order_data_filtered %>% 
  count(shopid) %>% 
  group_by(shopid) %>% 
  summarise(userid = paste(userid, collapse = '&'))
head(order_subs1)

`summarise()` ungrouping output (override with `.groups` argument)



shopid,userid
<int>,<chr>
10159,214988798
10287,103289694&198097381&214226569
10402,77819
10536,672345
26133,32449911
29583,214495220&215275495


List of different shopids

In [19]:
shopid_dist<-order_data %>% 
  select(shopid) %>% 
  distinct(shopid,.keep_all = T)
head(shopid_dist)
nrow(shopid_dist) #18770 distinct shopids

,shopid
,<int>
1,93950878
2,156423439
3,173699291
4,63674025
5,127249066
6,173811070


[1] 18770

Join list of all shopids and suspected order brushing shops

In [17]:
shopid_dist2<-shopid_dist %>% 
  full_join(order_subs1,by='shopid')
head(shopid_dist2)

,shopid,userid
,<int>,<chr>
1,93950878,NA
2,156423439,NA
3,173699291,NA
4,63674025,NA
5,127249066,NA
6,173811070,NA


Convert missing values with "0" for shops that are not suspected to commit order brushing. This will be the **final results**

In [18]:
shopid_dist2$userid<-ifelse(is.na(shopid_dist2$userid),0,shopid_dist2$userid)
head(shopid_dist2)

,shopid,userid
,<int>,<chr>
1,93950878,0
2,156423439,0
3,173699291,0
4,63674025,0
5,127249066,0
6,173811070,0


Score: 0.9262 (Rank 85)
;Problems: Can't convert "Time" class into time. Only 1 day transactions are suspected. Attempted to find difference between two times, but can't detect order brushing with different hour (e.g. 19:59 to 20:10).